In [1]:
PATH_BASE = 'Data/'
INDEX_KNOWLEDGE_BASE = "abstracts_full_en"
DO_PREPROCESSING = False
ABSTRACT_COLUMN = 10
ANSWER_COLUMN = 6
QUESTION_COLUMN = 2

In [2]:
import pandas as pd


pira_train = pd.read_csv(PATH_BASE + "train.csv").values.tolist()
pira_test = pd.read_csv(PATH_BASE + "test.csv").values.tolist()
pira_val = pd.read_csv(PATH_BASE + "validation.csv").values.tolist()

pira_dataset = pira_train+pira_test+pira_val

abstracts = []
temp = []
for i in range(len(pira_dataset)):
    if pira_dataset[i][10] not in temp:
        abstracts.append([pira_dataset[i][10], len(abstracts)+1])
        temp.append(pira_dataset[i][10])
del temp 
 
for i in range(len(pira_dataset)):
    for j in range(len(abstracts)):
        if pira_dataset[i][10] == abstracts[j][0]:
            pira_dataset[i].append(abstracts[j][1])
            
dicts = []
for line in abstracts:
    dicts.append({'content' : line[0], 'meta' : {'idarticle': line[1]}})



675

In [3]:

from haystack.preprocessor import PreProcessor
if DO_PREPROCESSING:

    processor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=True,
        split_by="word",
        split_length=NUMBER_OF_WORDS,
        split_respect_sentence_boundary=False,
        split_overlap=0
    )
    docs = []
    cont=0

    for dict1 in dicts:
        if cont %100 ==0:
            print(cont)
        cont+=1
        doc = processor.process(dict1)
        docs = docs+doc
else:
    docs = dicts

/root/anaconda3/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(
WARNING - haystack -  Object 'PreProcessor' is imported through a deprecated path. Please check out the docs for the new import path.
WARNING - haystack -  Object 'PreProcessor' is imported through a deprecated path. Please check out the docs for the new import path.


In [4]:
import os
os.remove("faiss_document_store.db") 

In [5]:
from haystack.document_store import FAISSDocumentStore

document_store = FAISSDocumentStore(similarity="dot_product", index=INDEX_KNOWLEDGE_BASE)

document_store.write_documents(docs)


from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)
# Important: 
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation. 
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

document_store.save("DPR_Store/DPR_Store")

WARNING - haystack -  Object 'FAISSDocumentStore' is imported through a deprecated path. Please check out the docs for the new import path.


Writing Documents:   0%|          | 0/675 [00:00<?, ?it/s]

INFO - haystack.modeling.utils -  Using devices: CUDA:0, CUDA:1
INFO - haystack.modeling.utils -  Number of GPUs: 2
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-ctx_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx

Updating Embedding:   0%|          | 0/675 [00:00<?, ? docs/s]

/root/anaconda3/lib/python3.9/site-packages/haystack/modeling/data_handler/dataset.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180487213/work/torch/csrc/utils/tensor_new.cpp:201.)
  cur_tensor = torch.tensor([sample[t_name] for sample in features], dtype=torch.long)


Create embeddings:   0%|          | 0/688 [00:00<?, ? Docs/s]

In [6]:
from haystack.pipelines import DocumentSearchPipeline

pipeline = DocumentSearchPipeline(retriever=retriever)


In [7]:
question = "Brazi pre salt ?"


result = pipeline.run(
    query=question,
    params={
        "Retriever": {
            "top_k": 5,
        }
    }
)



In [8]:
result

{'documents': [<Document: {'content': 'Pre-salt carbonate reservoirs are located offshore Brazil at Santos, Campos and Espírito Santo Basins, reaching depths up to 7000 m (22966 ft) and water depths of approximately 2200 m (7218 ft). Most of those reservoirs present very high permeability, resulting in extraordinary oil production rates, imposing, consequently, an onerous task on injector wells to maintain pressure and reservoir mass balance. Part of these injector wells are located in scenarios of inferior permo-porous characteristic and still have their injection rates limited in order to avoid fracture propagation in the reservoirs. To improve the geomechanical models, a series of field tests were carried out to estimate minimum in situ stress and fracture propagation pressure in both reservoir and cap rock. The tests results allowed a change in the criterion for definition of the maximum injection pressure in injector wells. The objective of this work is to present the tests result

In [10]:
import ast
import pandas as pd
def get_DPR_acc(questions, K_Values):
    maxK = max(K_Values)
    cont = 0
    ids = []
    for line in questions:
        ids.append([])
        result = pipeline.run(query=line[QUESTION_COLUMN],params={"Retriever": {"top_k": maxK}})
        document_dict = ast.literal_eval(str(result["documents"]).replace("<Document: ","").replace("'}>","'}"))
        for i in range(len(document_dict)):
            ids[cont].append(int(document_dict[i]["meta"]["idarticle"]))
        cont+=1
    corrects = []
    accuracies = []
    for j in range(len(K_Values)):
        corrects.append(0)
        cont = 0
        for line in questions:
            if int(line[len(line)-1]) in ids[cont][:K_Values[j]]:
                corrects[j]+=1
            cont+=1
        accuracies.append(corrects[j]/len(questions))
    return(accuracies)

accs = []
#    Ks = [1, 2, 5, 10, 25, 50, 100]
Ks = range(1,101)
accs = get_DPR_acc( pira_test, Ks)
for i in range(len(Ks)):
    print("accuracy for K = " + str(Ks[i]) + " -- is =" + str(accs[i]))
    
df_accs = pd.DataFrame(accs)
df_accs.to_csv("DPR_results.csv")

accuracy for K = 1 -- is =0.3656387665198238
accuracy for K = 2 -- is =0.44933920704845814
accuracy for K = 3 -- is =0.4845814977973568
accuracy for K = 4 -- is =0.5242290748898678
accuracy for K = 5 -- is =0.5550660792951542
accuracy for K = 6 -- is =0.5859030837004405
accuracy for K = 7 -- is =0.5903083700440529
accuracy for K = 8 -- is =0.5947136563876652
accuracy for K = 9 -- is =0.6299559471365639
accuracy for K = 10 -- is =0.6475770925110133
accuracy for K = 11 -- is =0.6519823788546255
accuracy for K = 12 -- is =0.6563876651982379
accuracy for K = 13 -- is =0.6740088105726872
accuracy for K = 14 -- is =0.6960352422907489
accuracy for K = 15 -- is =0.7092511013215859
accuracy for K = 16 -- is =0.7180616740088106
accuracy for K = 17 -- is =0.7180616740088106
accuracy for K = 18 -- is =0.7180616740088106
accuracy for K = 19 -- is =0.7312775330396476
accuracy for K = 20 -- is =0.7533039647577092
accuracy for K = 21 -- is =0.7577092511013216
accuracy for K = 22 -- is =0.7665198237885